In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [2]:
# Parameters (make sure these are correct)
subscr_id = "2be2dbbb-7c00-4b84-8fba-4b0199750c01" 
resc_name = "marterkist5"
worksp_name = "detectron2-workspace"

# Get a handle to the workspace
ml_client = MLClient(
    credential = credential,
    subscription_id = subscr_id,
    resource_group_name = resc_name,
    workspace_name = worksp_name,
)

# ws 
# from azureml.core import Workspace
# ws = Workspace(subscr_id, resc_name, worksp_name)

In [3]:
### First run az login in a command prompt to make sure you have authenticated using the Azure CLI

from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "cpu-cluster32GB"
cpu_size = "STANDARD_E4S_V3"

gpu_compute_target = "gpu-cluster28GB"
gpu_size = "STANDARD_NC4AS_T4_V3"

def create_compute(target, size):
    try:
        # let's see if the compute target already exists
        cluster = ml_client.compute.get(target)
        print(
            f"You already have a cluster named {target}, we'll reuse it as is."
        )

    except Exception:
        print("Creating a new compute target...")

        # Let's create the Azure ML compute object with the intended parameters
        cluster = AmlCompute(
            # Name assigned to the compute cluster
            name=target,
            # Azure ML Compute is the on-demand VM service
            type="amlcompute",
            # VM Family
            size=size,
            # Minimum running nodes when there is no job running
            min_instances=0,
            # Nodes in cluster
            max_instances=4,
            # How many seconds will the node running after the job termination
            idle_time_before_scale_down=180,
            # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
            tier="Dedicated",
        )

        # Now, we pass the object to MLClient's create_or_update method
        cluster = ml_client.begin_create_or_update(cluster).result()

    print(
        f"AMLCompute with name {cluster.name} is created, the compute size is {cluster.size}"
    )
    
    return target
    
compute_target = create_compute(cpu_compute_target, cpu_size)

You already have a cluster named cpu-cluster32GB, we'll reuse it as is.
AMLCompute with name cpu-cluster32GB is created, the compute size is STANDARD_E4S_V3


In [4]:
# For job
env_name = "yolov8-env@latest"

# Or
# from azureml.core import Environment
# mycustomenv = Environment(name="detectron2-env@latest")

In [16]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential
import os

# Load from local

# Load from local
root_path = r"C:/Users/augus/Documents/Studie/MGI/Internship/detection-segmentation/Data/Ready/test"
data_set = "10m/0bg" # Folder in rootpath containing to be used images, masks, annotations (xm/ybg) 
data_path = os.path.join(root_path, data_set) 

data_path = r"C:/Users/augus/Documents/Studie/MGI/Internship/detection-segmentation/Data/Ready/test" #/YOLO"

# Load from data asset 
# train_data_asset = ml_client.data.get("test_dataasset", version="1") # Split into train and vali in script
# data_path = train_data_asset.id

In [24]:
from azure.ai.ml import command
from azure.ai.ml import Input
import os
import json

# Set job and run
job = command(
     code=".", 
     command='python yolov8_train.py --traindata ${{inputs.train_data}}',
     inputs={
         "train_data": Input(path=data_path,
             type=AssetTypes.URI_FOLDER,
             mode=InputOutputModes.RO_MOUNT # Mount (.RO_MOUNT) or download (.DOWNLOAD)
         ),
         "epochs": 5,
         "batch": 3,
         "lr0": 0.00025, # the real "batch size"
         "roi_batch_size": 16, # the ROI head "batch size"
     },
     compute=compute_target,
     environment=env_name,
     outputs={},  # Add any required outputs here
#      description=description
 )

ml_client.jobs.create_or_update(job)

Uploading yolov8 (0.04 MBs): 100%|############################################| 37458/37458 [00:01<00:00, 35054.86it/s]


Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'C:/Users/augus/Documents/Studie/MGI/Internship/detection-segmentation/Data/Ready/test' 'https://detectron2work1228828607.blob.core.windows.net/azureml-blobstore-03504c2f-c8b0-4ff1-b795-1b7f7d4de4c9/LocalUpload/c5168b5238cf5f66ca45b47db385964e/test' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.


Experiment,Name,Type,Status,Details Page
yolov8,dreamy_sponge_wjsrb8bwj2,command,Starting,Link to Azure Machine Learning studio


In [ ]:
from azure.ai.ml.sweep import Uniform

# we will reuse the command_job created before. we call it as a function so that we can apply inputs
job_for_sweep = job(
    learning_rate=Uniform(min_value=0.0005, max_value=0.005),
    momentum=Uniform(min_value=0.9, max_value=0.99),
)